In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from datetime import datetime
import json
import torch as t
import pandas as pd


In [2]:
# Suppress warning messages
from transformers.utils import logging
logging.set_verbosity(40)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [3]:
# Program variables
max_iterations = 10
conversation_history = list()
model_id = "codellama/CodeLlama-7b-Instruct-hf"
filename = f"{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.txt"


In [4]:
device = "cuda:0" if t.cuda.is_available() else "cpu"

In [5]:
device

'cuda:0'

In [6]:
cache_dir ="../datax/models"

In [7]:
# Load model
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, cache_dir=cache_dir, device_map=device, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_dir, padding_side="left")
streamer = TextStreamer(tokenizer, skip_prompt=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model.to(device)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
      (1): LlamaDecoderLayer(
        (self_attn): LlamaAtten

In [9]:
p1_template= "The following snippet is a exploitable code with a score of {}  {}"

In [10]:
p2_template = "What is the score for the following snippet?  {}"

In [8]:
prompt1 = '''The following snippet is a exploitable code with a score of 4.5
	dev = usb_get_intfdata(interface);
	if (!dev) {
		retval = -ENODEV;
		goto exit;
	}

	/* increment our usage count for the device */
	kref_get(&dev->kref);

	/* save our object in the file's private structure */
	mutex_lock(&dev->io_mutex);
	file->private_data = dev;
	mutex_unlock(&dev->io_mutex);'''

In [9]:
prompt2 = '''The following snippet has a score of 7.5: int i;
char inLine[64];
cin >> inLine;
i = atoi (inLine);
sleep(i);'''

In [10]:
prompt3 = '''What is the score for the following snippet? 
int main(int argc, char *argv[])
{
	rc = SQLConnect(Example.ConHandle, argv[0], SQL_NTS,
	(SQLCHAR *) "", SQL_NTS, (SQLCHAR *) "", SQL_NTS);
} '''

In [11]:
MESSAGES_LENGTH = 1

In [12]:
val_data = pd.read_csv("../datax/big-vul/train.csv")

In [13]:
val_data.head()

,index,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,Confidentiality,...,parentID,patch,project,project_after,project_before,target,vul_func_with_fix,processed_func,flaw_line,flaw_line_index
0,12473,NaN,Remote,Not required,NaN,CVE-2014-3508,https://www.cvedetails.com/cve/CVE-2014-3508/,CWE-200,Medium,Partial,...,17160033765480453be0a41335fa6b833691c049,"@@ -471,11 +471,12 @@ int OBJ_obj2txt(char *bu...",openssl,https://git.openssl.org/gitweb/?p=openssl.git;...,https://git.openssl.org/gitweb/?p=openssl.git;...,0,const char *OBJ_nid2sn(int n)\n\t{\n\tADDED_OB...,const char *OBJ_nid2sn(int n)\n\t{\n\tADDED_OB...,NaN,NaN
1,24444,NaN,Local,Not required,NaN,CVE-2011-4080,https://www.cvedetails.com/cve/CVE-2011-4080/,CWE-264,High,Complete,...,NaN,"@@ -170,6 +170,11 @@ static int proc_taint(str...",linux,bfdc0b497faa82a0ba2f9dddcf109231dd519fcc,cb16e95fa2996743a6e80a665ed2ed0590bd38cf,0,void register_sysctl_root(struct ctl_table_roo...,void register_sysctl_root(struct ctl_table_roo...,NaN,NaN
2,111436,NaN,Remote,Not required,Partial,CVE-2012-2875,https://www.cvedetails.com/cve/CVE-2012-2875/,NaN,Medium,Partial,...,NaN,"@@ -4058,11 +4058,6 @@ bool WebPage::touchEven...",Chrome,d345af9ed62ee5f431be327967f41c3cc3fe936a,e261bb8e47a6a9fdd1d26fd52b1538c5c9bcb122,0,WebPagePrivate::~WebPagePrivate()\n{\n // H...,WebPagePrivate::~WebPagePrivate()\n{\n m_we...,NaN,NaN
3,1314,NaN,Remote,Not required,Complete,CVE-2009-3604,https://www.cvedetails.com/cve/CVE-2009-3604/,CWE-399,Medium,Complete,...,75c3466ba2e4980802e80b939495981240261cd5,"@@ -216,6 +216,28 @@ void *gmallocn_checkoverf...",poppler,https://cgit.freedesktop.org/poppler/poppler/t...,https://cgit.freedesktop.org/poppler/poppler/t...,0,"char *gstrndup(const char *s, size_t n) {\n c...","char *gstrndup(const char *s, size_t n) {\n c...",NaN,NaN
4,88406,NaN,Remote,Not required,NaN,CVE-2019-15164,https://www.cvedetails.com/cve/CVE-2019-15164/,CWE-918,Low,NaN,...,NaN,"@@ -156,6 +156,8 @@ static int rpcapd_recv(SOC...",libpcap,33834cb2a4d035b52aa2a26742f832a112e90a0a,484d60cbf7ca4ec758c3cbb8a82d68b244a78d58,0,"daemon_AuthUserPwd(char *username, char *passw...","daemon_AuthUserPwd(char *username, char *passw...",NaN,NaN


In [14]:
filtered_val = val_data[val_data['func_before'].str.len() < 100]


In [15]:
test_data = pd.read_csv("../datax/big-vul/test.csv")

In [16]:
filtered_test = test_data[test_data['func_before'].str.len() < 100]


In [17]:
test_data.head()

,index,Access Gained,Attack Origin,Authentication Required,Availability,CVE ID,CVE Page,CWE ID,Complexity,Confidentiality,...,parentID,patch,project,project_after,project_before,target,vul_func_with_fix,processed_func,flaw_line,flaw_line_index
0,73752,NaN,Remote,Not required,Complete,CVE-2016-6561,https://www.cvedetails.com/cve/CVE-2016-6561/,CWE-476,Low,NaN,...,NaN,"@@ -21,6 +21,7 @@\n /*\n * Copyright (c) 2007...",illumos-gate,6d1c73b5858fefc6161c7d686345f0dc887ea799,516627f338a630bcf9806a91aa873bbbae9a2fac,0,smb_ofile_delete(void *arg)\n{\n\tsmb_tree_t\t...,smb_ofile_delete(void *arg)\n{\n\tsmb_tree_t\t...,NaN,NaN
1,54196,NaN,Local,Not required,Complete,CVE-2016-3138,https://www.cvedetails.com/cve/CVE-2016-3138/,NaN,Low,NaN,...,NaN,"@@ -1179,6 +1179,9 @@ static int acm_probe(str...",linux,8835ba4a39cf53f705417b3b3a94eb067673f2c9,0b818e3956fc1ad976bee791eadcbb3b5fec5bfd,0,static inline int acm_set_control(struct acm *...,static inline int acm_set_control(struct acm *...,NaN,NaN
2,169124,NaN,Remote,Not required,NaN,CVE-2018-6145,https://www.cvedetails.com/cve/CVE-2018-6145/,CWE-79,Medium,NaN,...,NaN,"@@ -82,13 +82,6 @@ static bool TokenExitsForei...",Chrome,133bc5c262b2555af223263452e9875a95db9eb7,1e8327c88920544f1503004b4e32850c935d4efb,0,HTMLTreeBuilderSimulator::State HTMLTreeBuilde...,HTMLTreeBuilderSimulator::State HTMLTreeBuilde...,NaN,NaN
3,109551,NaN,Remote,Not required,Partial,CVE-2012-5135,https://www.cvedetails.com/cve/CVE-2012-5135/,CWE-399,Low,Partial,...,NaN,"@@ -713,7 +713,8 @@ PrintWebViewHelper::PrintW...",Chrome,b755ebba29dd405d6f1e4cf70f5bc81ffd33b0f6,7b688dec9fa8ab42a4933e381ad9aeb63413139b,0,int PrintWebViewHelper::PrintPreviewContext::t...,int PrintWebViewHelper::PrintPreviewContext::t...,NaN,NaN
4,78906,NaN,Local,Not required,Complete,CVE-2018-16276,https://www.cvedetails.com/cve/CVE-2018-16276/,CWE-20,Low,Complete,...,NaN,"@@ -396,35 +396,24 @@ static ssize_t yurex_rea...",linux,f1e255d60ae66a9f672ff9a207ee6cd8e33d2679,bba57eddadda936c94b5dccf73787cb9e159d0a5,0,"static int yurex_open(struct inode *inode, str...","static int yurex_open(struct inode *inode, str...",NaN,NaN


In [18]:
test_data.columns.tolist()

['index',
 'Access Gained',
 'Attack Origin',
 'Authentication Required',
 'Availability',
 'CVE ID',
 'CVE Page',
 'CWE ID',
 'Complexity',
 'Confidentiality',
 'Integrity',
 'Known Exploits',
 'Publish Date',
 'Score',
 'Summary',
 'Update Date',
 'Vulnerability Classification',
 'add_lines',
 'codeLink',
 'commit_id',
 'commit_message',
 'del_lines',
 'file_name',
 'files_changed',
 'func_after',
 'func_before',
 'lang',
 'lines_after',
 'lines_before',
 'parentID',
 'patch',
 'project',
 'project_after',
 'project_before',
 'target',
 'vul_func_with_fix',
 'processed_func',
 'flaw_line',
 'flaw_line_index']

In [19]:
random_row = test_data.sample(n=1)
text = random_row['func_before'].values[0]
random_row['Score'].values[0]

9.3

In [20]:
text

'void CL_Init( void ) {\n\tCom_Printf( "----- Client Initialization -----\\n" );\n\n\tCon_Init ();\n\n\tif(!com_fullyInitialized)\n\t{\n\t\tCL_ClearState();\n\t\tclc.state = CA_DISCONNECTED;\t// no longer CA_UNINITIALIZED\n\t\tcl_oldGameSet = qfalse;\n\t}\n\n\tcls.realtime = 0;\n\n\tCL_InitInput ();\n\n\tcl_noprint = Cvar_Get( "cl_noprint", "0", 0 );\n#ifdef UPDATE_SERVER_NAME\n\tcl_motd = Cvar_Get ("cl_motd", "1", 0);\n#endif\n\n\tcl_timeout = Cvar_Get ("cl_timeout", "200", 0);\n\n\tcl_timeNudge = Cvar_Get ("cl_timeNudge", "0", CVAR_TEMP );\n\tcl_shownet = Cvar_Get ("cl_shownet", "0", CVAR_TEMP );\n\tcl_showSend = Cvar_Get ("cl_showSend", "0", CVAR_TEMP );\n\tcl_showTimeDelta = Cvar_Get ("cl_showTimeDelta", "0", CVAR_TEMP );\n\tcl_freezeDemo = Cvar_Get ("cl_freezeDemo", "0", CVAR_TEMP );\n\trcon_client_password = Cvar_Get ("rconPassword", "", CVAR_TEMP );\n\tcl_activeAction = Cvar_Get( "activeAction", "", CVAR_TEMP );\n\n\tcl_timedemo = Cvar_Get ("timedemo", "0", 0);\n\tcl_timedemoLog

In [21]:
def build_messages():
    messages = []
    indexes = []
    gt = None
    for i in range(MESSAGES_LENGTH):
        random_row = filtered_val.sample(n=1)
        text = random_row['func_before'].values[0]
        score = random_row['Score'].values[0]
        message = p1_template.format(score, text)
        indexes.append(random_row['index'].values[0])
        messages.append(message)
    random_row = filtered_test.sample(n=1)
    text = random_row['func_before'].values[0]
    gt = random_row['Score'].values[0]
    message = p2_template.format(text)
    indexes.append(random_row['index'].values[0])
    messages.append(message)
    return messages, indexes, gt

In [22]:
messages, indexes, gt_score = build_messages()

In [23]:
indexes

[120304, 1267]

In [24]:
# Limit maximum iterations for conversation
def generate_prediction(messages):
    conversation_history = list()

    for message in messages:

        conversation_history.append({"role": "user", "content": message})
        conversation_history.append({"role": "system", "content": ""})
        # Convert conversational history into chat template and tokenize
        inputs = tokenizer.apply_chat_template(conversation_history, return_tensors="pt", return_attention_mask=False).to(device)

        # Generate output
        generated_ids = model.generate(inputs,
            #streamer=streamer,
            max_new_tokens=20,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature= 0.9,
            pad_token_id=tokenizer.eos_token_id
        )

        # Get complete output from model including input prompt
        output = tokenizer.batch_decode(generated_ids)[0]

        # Filter only new output information using '</s>' delimiter, then strip starting and trailing whitespace
        output_filtered = output.split('[/INST]')[-1].strip()

        # Update conversation history with the latest output
        conversation_history[-1]["content"] = output_filtered

    return conversation_history

        # Capture input before start of next iteration
        #capture_input()

In [25]:
def generate_trials():
    conversations = list()
    messages, indexes, gt_score = build_messages()
    for i in range(max_iterations):
        conversation = generate_prediction(messages)
        conversations.append(conversation)
    return conversations, indexes, gt_score
        
    

In [26]:
conversations, indexes, gt_score = generate_trials()

In [67]:
def save_conversations(conversation_history):
    # Save entire conversation history to text file for debugging or use for loading conversational context
    with open(filename, 'w') as f:
        json.dump(conversation_history, f, ensure_ascii=False, indent=4)

In [27]:
SAMPLES = 100

In [28]:
result = []
for i in range(SAMPLES):
    conversations, indexes, gt_score = generate_trials()
    conversations.append({"indexes":indexes, "gt_score":gt_score})
    result.append(conversations)

In [ ]:
filename = f"{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_100_10_codeLlama7b.txt"
save_conversations(result)

In [ ]:


# Load conversational history from a previous context file
context_filename = "./*.txt"
with open(context_filename, 'r') as f:
     data = json.load(f)
     conversation_history = data


